In [1]:
from DB import DataFrameHandler
import pandas as pd

pd.options.mode.chained_assignment = None

def getBenchmark(idx):
    idxCode = {"KOSPI":1, "KOSDAQ":2, "RATE":3}
    if idx.upper() in idxCode.keys():
        benchmark = DataFrameHandler.findItems('Daily_Index', condition=f"IDX_CODE={idxCode[idx.upper()]}", orderBy='WORK_DT')
        benchmark= benchmark[['WORK_DT','VALUE']].reset_index(drop=True)
    return benchmark

In [2]:
workDate = DataFrameHandler.findItems('work_date', condition="work_dt >= to_date('2023/09/01','yyyy/mm/dd')", orderBy= 'date_label')
orderRequest = DataFrameHandler.findItems('order_request', columns=['odr_id','acc_id','isin','work_dt','order_price','order_volume', 'order_type', 'order_result'], condition="acc_id = (select acc_id from account where one_id = 'hoomba1234')", orderBy="odr_id")
stocks = orderRequest['ISIN'].unique()
stockString = "'" + "', '".join(stocks) + "'"
ohlcv = DataFrameHandler.findItems('ohlcv', columns = ['ISIN', 'WORK_DT', 'CLOSE'], condition=f"work_dt >= to_date('2023/09/01','yyyy/mm/dd') and ISIN in ({stockString})", orderBy="WORK_DT")
values = pd.DataFrame([])
for ticker in stocks:
    stock = orderRequest[orderRequest['ISIN'] == ticker]
    stock['VOLUME_CHANGE'] = stock['ORDER_VOLUME'] * (stock['ORDER_TYPE'] * -2 + 1)
    stock = stock.groupby('WORK_DT').agg({'VOLUME_CHANGE': 'sum'}).reset_index()
    stock = pd.merge(workDate, stock, how='outer')
    stock['VOLUME_CHANGE'].fillna(0, inplace=True)
    stock["BALANCE_VOLUME"] = stock['VOLUME_CHANGE'].cumsum()
    stock = stock[['WORK_DT', 'BALANCE_VOLUME']]
    stock = pd.merge(stock, ohlcv[ohlcv['ISIN'] == ticker], how='inner')
    stock['VALUE'] = stock['BALANCE_VOLUME'] * stock['CLOSE']
    stock = stock[['WORK_DT', 'ISIN','VALUE']]
    values = pd.concat([values, stock])
values = values.groupby('WORK_DT').agg({'VALUE': 'sum'}).reset_index()
sellRequest = orderRequest[orderRequest['ORDER_TYPE']==1]
sellRequest['SELL_ORDER_AMOUNT'] = sellRequest['ORDER_PRICE'] * sellRequest['ORDER_VOLUME']
sellRequest = sellRequest.groupby('WORK_DT').agg({'SELL_ORDER_AMOUNT': 'sum'}).reset_index()
sellRequest = pd.merge(workDate, sellRequest, how='outer')
sellRequest['SELL_ORDER_AMOUNT'].fillna(0, inplace=True)
sellRequest['SELL_AMOUNT'] = sellRequest['SELL_ORDER_AMOUNT'].cumsum()
buyRequest = orderRequest[orderRequest['ORDER_TYPE']==0]
buyRequest['BUY_ORDER_AMOUNT'] = buyRequest['ORDER_PRICE'] * buyRequest['ORDER_VOLUME']
buyRequest = buyRequest.groupby('WORK_DT').agg({'BUY_ORDER_AMOUNT': 'sum'}).reset_index()
buyRequest = pd.merge(workDate, buyRequest, how='outer')
buyRequest['BUY_ORDER_AMOUNT'].fillna(0, inplace=True)
buyRequest['BUY_AMOUNT'] = buyRequest['BUY_ORDER_AMOUNT'].cumsum()
portfolio = buyRequest[['WORK_DT']]
portfolio[['SELL_ORDER_AMOUNT', 'PROFIT']] = sellRequest[['SELL_ORDER_AMOUNT', 'SELL_AMOUNT']]
portfolio[['BUY_ORDER_AMOUNT', 'BUY_AMOUNT']] = buyRequest[['BUY_ORDER_AMOUNT', 'BUY_AMOUNT']]
portfolio['ACC_ID'] = orderRequest['ACC_ID'][0]
portfolio['VALUE_AMOUNT'] = values['VALUE']
portfolio = portfolio[['ACC_ID', 'WORK_DT', 'BUY_AMOUNT', 'VALUE_AMOUNT', 'PROFIT']]
portfolio

,ACC_ID,WORK_DT,BUY_AMOUNT,VALUE_AMOUNT,PROFIT
0,41071304394837,2023-09-01,3935000.0,3950000.0,0.0
1,41071304394837,2023-09-04,5100000.0,5177000.0,0.0
2,41071304394837,2023-09-05,5100000.0,5159000.0,0.0
3,41071304394837,2023-09-06,5100000.0,5177000.0,0.0
4,41071304394837,2023-09-07,5100000.0,5195000.0,0.0
5,41071304394837,2023-09-08,6150000.0,6229000.0,0.0
6,41071304394837,2023-09-11,6150000.0,6292000.0,0.0
7,41071304394837,2023-09-12,6600000.0,6747000.0,0.0
8,41071304394837,2023-09-13,6600000.0,5987000.0,824000.0
9,41071304394837,2023-09-14,8319000.0,7241500.0,1374000.0


In [3]:
benchmark = getBenchmark("kospi")
benchmark = benchmark[benchmark['WORK_DT'].isin(portfolio['WORK_DT'])].reset_index()
benchmark['BENCHMARK'] = (benchmark['VALUE'] - benchmark['VALUE'].iloc[0]) / benchmark['VALUE'].iloc[0] * 100
portfolio['BENEFIT'] = (portfolio['VALUE_AMOUNT'] + portfolio['PROFIT'] - portfolio['BUY_AMOUNT'])/portfolio['BUY_AMOUNT'] * 100
portfolio = pd.merge(portfolio, benchmark, how= 'inner')
portfolio['DIFF'] = portfolio['BENEFIT'] - portfolio['BENCHMARK']
portfolio['DIFF_AVG'] = 0.0
portfolio['DIFF_STD'] = 0.0

for i in range(0, len(portfolio)):
    diff_values = portfolio['DIFF'][:i + 1]
    portfolio.at[i, 'DIFF_AVG'] = diff_values.mean()
    portfolio.at[i, 'DIFF_STD'] = diff_values.std()
    
portfolio["SHARP"] = portfolio["DIFF_AVG"] / portfolio["DIFF_STD"]
portfolio.fillna(0, inplace=True)

portfolio['BM_COV'] = 0.0
portfolio['BM_VAR'] = 0.0

for i in range(0, len(portfolio)):
    bm = portfolio['BENCHMARK'][:i + 1]
    be = portfolio['BENEFIT'][:i + 1]
    portfolio.at[i, 'BM_COV'] = bm.cov(be)
    portfolio.at[i, 'BM_VAR'] = bm.var()

portfolio["BETA"] = portfolio["BM_COV"] / portfolio["BM_VAR"]
portfolio.fillna(0, inplace=True)
portfolio["TREYNOR"] = portfolio["DIFF_AVG"] / portfolio["BETA"]
portfolio.fillna(0, inplace=True)
portfolio = portfolio[['ACC_ID', 'WORK_DT', 'BUY_AMOUNT', 'VALUE_AMOUNT', 'PROFIT', 'SHARP', 'BETA', 'TREYNOR']]
portfolio = portfolio.replace([float('inf'), float('-inf')], 0)
portfolio

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/nanops.py:1671: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


,ACC_ID,WORK_DT,BUY_AMOUNT,VALUE_AMOUNT,PROFIT,SHARP,BETA,TREYNOR
0,41071304394837,2023-09-01,3935000.0,3950000.0,0.0,0.000000,0.000000,0.000000
1,41071304394837,2023-09-04,5100000.0,5177000.0,0.0,2.414574,1.388401,0.388257
2,41071304394837,2023-09-05,5100000.0,5159000.0,0.0,2.994028,1.269714,0.397606
3,41071304394837,2023-09-06,5100000.0,5177000.0,0.0,1.438916,0.511062,1.486498
4,41071304394837,2023-09-07,5100000.0,5195000.0,0.0,1.237799,-0.189161,-5.819556
5,41071304394837,2023-09-08,6150000.0,6229000.0,0.0,1.434660,-0.135658,-9.108660
6,41071304394837,2023-09-11,6150000.0,6292000.0,0.0,1.525941,-0.267864,-5.327502
7,41071304394837,2023-09-12,6600000.0,6747000.0,0.0,1.526670,-0.426969,-3.886360
8,41071304394837,2023-09-13,6600000.0,5987000.0,824000.0,1.447719,-0.711121,-2.750489
9,41071304394837,2023-09-14,8319000.0,7241500.0,1374000.0,1.560265,-0.419831,-4.956621


In [4]:
DataFrameHandler.insertItems('portfolio', portfolio)